# Importing all the libraries

---------------------------------------------------------------------------------------------------------------------------

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

### Reading the data

---------------------------------------------------------------------------------------------------------------------------

In [2]:
PROJECT_DIR = r"C:\Users\Dell\Desktop\Machine Learning\FLight Price Predictions (AWS Sagemaker)"
DATA = "Data"

In [3]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA, file_name)
    return pd.read_csv(file_path)

In [4]:
flights = get_data('flight_price')
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [5]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


---------------------------------------------------------------------------------------------------------------------------
- The dataset contains 10,683 rows and 11 features
- Columns 'Route' and 'Total_Stops' have a missing value each
- The data type of some features is inappropriate
---------------------------------------------------------------------------------------------------------------------------

### Preliminary Analysis
---------------------------------------------------------------------------------------------------------------------------

- Check Data Types

In [6]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

- Check Duplicate Values

In [8]:
flights.duplicated().sum()

220

In [9]:
(flights.loc[flights.duplicated(keep = False)].sort_values(['Airline','Date_of_Journey','Source','Destination']))

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


#### Observations
     
   1. The type of `Date_of_journey` should be date
   2. The type of `Dep_time` should be time
   3. The type of `Arrival_Time` should be time
   4. The type of `Duration` should be numeric
   5. The type of `Total_Stops` should be numeric
   6. There are 220 duplicate values

### Detailed Analysis

---------------------------------------------------------------------------------------------------------------------------

***Airline***

In [10]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Some of the entries have inaccurate values 

***Date_of_journey***

In [11]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

- Date of journey is not in the Datetime format

***Source***

In [12]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

- No changes need to be done in this feature

***Destination***

In [13]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

- No changes need to be done in this feature

***Route***

In [14]:
flights.Route

0                    BLR → DEL
1        CCU → IXR → BBI → BLR
2        DEL → LKO → BOM → COK
3              CCU → NAG → BLR
4              BLR → NAG → DEL
                 ...          
10678                CCU → BLR
10679                CCU → BLR
10680                BLR → DEL
10681                BLR → DEL
10682    DEL → GOI → BOM → COK
Name: Route, Length: 10683, dtype: object

- There is no need of this column as this information can be obtained from Source, Destination and Stops

***Departure Time***

In [15]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [16]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]                     # Checking for any other special characters in this column
)

Series([], Name: Dep_Time, dtype: object)

- The Dep_time Column is not in the datetime format

In [17]:
    pd.to_datetime(flights.Dep_Time).dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

***Arrival Time***

In [18]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [19]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]                     # Checking for any other special characters in this column
    .str.split(" ", n = 1)
    .str.get(-1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

***Duration***

In [20]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [21]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains('m')]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [22]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains('h')]
)

6474    5m
Name: Duration, dtype: object

In [23]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- The index 6474 has duration of 5 min which is an error

In [24]:
(
    flights
    .Duration
    .drop(index = 6474)
    .str.split(" ", expand = True)
    .set_axis(["hours","minutes"], axis = 1)
    .assign(
        hours = lambda df_: (
            df_
            .hours
            .str.replace('h','')
            .astype(int)
            .mul(60)
        ),
        minutes = lambda df_: (
            df_
            .minutes
            .str.replace('m','')
            .fillna('0')
            .astype(int)
        )
    )
    .sum(axis = 1)
    .rename('duration')
    .to_frame()
)

,duration
0,170
1,445
2,1140
3,325
4,285
...,...
10678,150
10679,155
10680,180
10681,160


***Total Stops***

In [6]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [5]:
(
    flights
    .Total_Stops
    .str.replace('non-stop','0')
    .str.replace(' stops?','',regex = True)  # The question mark make the s optional so it will do the work for s and without s
    .dropna()
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0
1        2
2        2
3        1
4        1
        ..
10678    0
10679    0
10680    0
10681    0
10682    2
Name: Total_Stops, Length: 10682, dtype: int64

***Additonal Info***

In [27]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

- No need to modify them for now

### Cleaning Data 

----------------------------------------------------------------------------------------------------------------------------

In [9]:
def convert_to_minutes(ser):
    return(
        ser
        .str.split(" ", expand = True)
        .set_axis(["hours","minutes"], axis = 1)
        .assign(
            hours = lambda df_: (
                df_
                .hours
                .str.replace('h','')
                .astype(int)
                .mul(60)
            ),
            minutes = lambda df_: (
                df_
                .minutes
                .str.replace('m','')
                .fillna('0')
                .astype(int)
            )
        )
        .sum(axis = 1)
    )

In [10]:
def clean_data(df):
    return(
        df
        .drop(index = 6474)
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include = "O").columns            # Stripping empty spaces
        })
        .rename(columns = str.lower)                                      # Renaming column header to lower case
        .assign(
            airline = lambda df_: (                                       # Cleaning the Airlines name 
                df_
                .airline
                .str.replace(' Premium economy', '')
                .str.replace(' Business', '')
                .str.title()
            ),
            date_of_journey = lambda df_: (                               # Changing the Datatype of Date_of_journey
                pd.to_datetime(df_.date_of_journey, dayfirst = True)
            ),
            dep_time = lambda df_: (
                pd.to_datetime(df_.dep_time).dt.time                      # Changing the Dep_time to date time format
            ),
            arrival_time = lambda df_: (
                pd.to_datetime(df_.arrival_time).dt.time                  # Changing the Arrival_Time to date time format
            ),
            duration = lambda df_ : df_.duration.pipe(convert_to_minutes), # Converting the Duration into minutes
            total_stops = lambda df_ : (
                df_
                .total_stops
                .str.replace('non-stop','0')
                .str.replace(' stops?','',regex = True)
                .dropna()
                .pipe(lambda ser: pd.to_numeric(ser))  
            ),
            additional_info = lambda df_ : df_.additional_info.replace('No info', 'No Info') 
        )
        .drop(columns = 'route')                                          # Dropping the Route Column
    )

In [11]:
flights_cleaned = clean_data(flights)
flights_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


### Split the Data

----------------------------------------------------------------------------------------------------------------------------

In [12]:
X = flights_cleaned.drop(columns = 'price')
y = flights_cleaned['price']

In [13]:
X_,X_test,y_,y_test = train_test_split(X, y, test_size =0.2, random_state =42)
X_train,X_val,y_train,y_val = train_test_split(X_, y_, test_size =0.2, random_state =42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6836, 9) (6836,)
(1709, 9) (1709,)
(2137, 9) (2137,)


### Export the Data

----------------------------------------------------------------------------------------------------------------------------

In [14]:
def export_data(X,y,name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA, file_name)
    
    X.join(y).to_csv(file_path, index = False)
    
    return pd.read_csv(file_path).head()

In [15]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-03,Delhi,Cochin,02:15:00,04:25:00,1570,1.0,No Info,17024
1,Vistara,2019-03-24,Kolkata,Banglore,07:10:00,18:45:00,695,1.0,No Info,16932
2,Spicejet,2019-04-09,Banglore,Delhi,09:30:00,12:20:00,170,0.0,No Info,4423
3,Indigo,2019-04-27,Banglore,Delhi,21:15:00,00:15:00,180,0.0,No Info,3943
4,Air India,2019-06-12,Delhi,Cochin,09:45:00,09:25:00,1420,1.0,No Info,7480


In [16]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-05-21,Delhi,Cochin,15:05:00,01:30:00,625,2.0,No Info,16655
1,Goair,2019-06-03,Delhi,Cochin,10:35:00,19:35:00,540,1.0,No Info,4959
2,Vistara,2019-05-09,Kolkata,Banglore,20:20:00,09:05:00,765,1.0,No Info,9187
3,Indigo,2019-05-24,Chennai,Kolkata,14:45:00,17:05:00,140,0.0,No Info,3858
4,Jet Airways,2019-05-21,Delhi,Cochin,22:50:00,04:25:00,335,1.0,In-flight meal not included,12898


In [17]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Kolkata,Banglore,16:30:00,12:00:00,1170,1.0,In-flight meal not included,8529
1,Jet Airways,2019-05-18,Banglore,Delhi,11:10:00,14:05:00,175,0.0,In-flight meal not included,5198
2,Multiple Carriers,2019-06-12,Delhi,Cochin,08:45:00,19:00:00,615,1.0,No Info,11789
3,Multiple Carriers,2019-04-24,Delhi,Cochin,07:10:00,16:10:00,540,1.0,In-flight meal not included,6093
4,Jet Airways,2019-03-27,Delhi,Cochin,15:05:00,04:25:00,800,1.0,No Info,12242
